In [1]:
!pip install catboost
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 14.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from  tqdm.notebook import tqdm
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, auc, roc_curve
from sklearn.metrics import precision_recall_curve
from catboost import CatBoostClassifier, Pool, cv
from catboost.utils import get_roc_curve
import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary

import tensorflow.keras as keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, SimpleRNN, GRU
from keras.layers import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import Cropping2D
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D
from keras.losses import BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras import regularizers
from keras.layers import BatchNormalization
from keras.metrics import Precision, Recall, AUC
from keras.utils import np_utils
from tensorflow.keras.utils import plot_model
from tensorflow_addons.metrics import F1Score
import graphviz, pydot, pydotplus
from scipy import sparse
from scipy.sparse.linalg import spsolve

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
STATE = 42
np.random.seed(STATE)

In [4]:
!unzip 01_healthy.zip
!unzip 02_sick.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: __MACOSX/01_healthy/._image_cortex_patient20022019_633nm_obj20_power100_1sec_сenter2900_step4um_place2_20200929_1_4.txt  
  inflating: 01_healthy/863_healthy.txt  
  inflating: 01_healthy/4754_healthy.txt  
  inflating: 01_healthy/4482_healthy.txt  
  inflating: 01_healthy/2299_healthy.txt  
  inflating: 01_healthy/115_healthy.txt  
  inflating: 01_healthy/2839_healthy.txt  
  inflating: 01_healthy/445_healthy.txt  
  inflating: 01_healthy/2656_healthy.txt  
  inflating: 01_healthy/3044_healthy.txt  
  inflating: 01_healthy/2306_healthy.txt  
  inflating: 01_healthy/3514_healthy.txt  
  inflating: 01_healthy/1903_healthy.txt  
  inflating: 01_healthy/3599_healthy.txt  
  inflating: 01_healthy/4590_healthy.txt  
  inflating: 01_healthy/557_healthy.txt  
  inflating: 01_healthy/image_cortex_patient15012020_633nm_obj20_power100_1sec_acc1_сenter2900_place1_dots20x15_10um_20210202_30.txt  
  inflating: __MACOSX

In [5]:
def standartise(data):
  return (data - data.mean(axis=1)[:, None]) / data.std(axis=1)[:, None]

In [6]:
def baseline_als(y, lam, p):
  L = len(y)
  D = sparse.diags([1,-2,1],[0,-1,-2], shape=(L,L-2))
  w = np.ones(L)
  for i in range(10):
    W = sparse.spdiags(w, 0, L, L)
    Z = W + lam * D.dot(D.transpose())
    z = spsolve(Z, w*y)
    w = p * (y > z) + (1-p) * (y < z)
  return z

In [7]:
def get_data(directory: str, etalon_columns: set = None) -> pd.DataFrame:
    df = pd.DataFrame()
    # Цикл для чтения каждого файла в каталоге
    for root, dirs, files in os.walk(directory):
        print(10)
        for i, filename in enumerate(files):
            print(filename)
            if filename.endswith(".txt"):
                file_path = os.path.join(directory, filename)
                # Чтение файла и добавление данных в DataFrame
                data = pd.read_csv(file_path, sep='\t')
                print('Загружен', file_path)

                # Оставляем первые 2 столбца, переименовываем
                data = data.iloc[:, :2]
                data.columns = ['#Wave', '#Intensity']

                # Удаление ненужных колонок       
                data = data.astype(float)
                data['#Wave'] = np.ceil(data['#Wave']).astype(int)

                # Нумеруем блоки по 1015 строк
                try:
                    data["Block"] = (data.index // 1015) + 1

                    # print(data.shape)

                    data_reshape = data.pivot(index='Block', columns='#Wave', values='#Intensity').reset_index(
                        drop=True)
                    # display(data_reshape)
                    if i == 0:
                        if etalon_columns == None:
                            etalon_columns = set(data_reshape.columns)
                            # print(set(data_reshape.columns))
                    # Все столбцы всех файлов получают частоты из 1 файла. !!!!
                    data_reshape.columns = etalon_columns
                except Exception as ex:
                    continue
                df = pd.concat([df, data_reshape], axis=0, ignore_index=True)

            # Проверка на совпадение частот
            #
            # if etalon_set == set(data_reshape.columns):
            #     # Разворачиваем данные, теперь будет 1015 столбцов с частотами. И добавляем в итоговый датафрейм
            #     df = pd.concat([df, data_reshape], axis = 0, ignore_index=True)
            # else:
            #     print('Частоты этого файла не совпадают с первым, пропускаем.')
            #     print(set(data_reshape.columns) - etalon_set)
            # if i == 1:
            #     break

    return df, etalon_columns

In [8]:
def model_metric(y_test, y_pred, y_pred_prob):
    # Расчет метрик на тестовом наборе
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    ###############################################################
    # Вывод результатов
    print(f"Accuracy: {accuracy:.3f}")
    print(f"Precision: {precision:.3f}")
    print(f"Recall: {recall:.3f}")
    print(f"F1-score: {f1:.3f}")
    
    return {'accuracy':accuracy, 'precision':precision, 'recall':recall, 'f1':f1}

In [9]:
directory = '01_healthy'
df_class0, etalon_columns = get_data(directory)
df_class0['labels'] = 0
df_class0.shape

Выходные данные были обрезаны до нескольких последних строк (5000).
4288_healthy.txt
Загружен 01_healthy/4288_healthy.txt
2616_healthy.txt
Загружен 01_healthy/2616_healthy.txt
26_healthy.txt
Загружен 01_healthy/26_healthy.txt
2357_healthy.txt
Загружен 01_healthy/2357_healthy.txt
1085_healthy.txt
Загружен 01_healthy/1085_healthy.txt
3983_healthy.txt
Загружен 01_healthy/3983_healthy.txt
2601_healthy.txt
Загружен 01_healthy/2601_healthy.txt
4450_healthy.txt
Загружен 01_healthy/4450_healthy.txt
1052_healthy.txt
Загружен 01_healthy/1052_healthy.txt
1664_healthy.txt
Загружен 01_healthy/1664_healthy.txt
2807_healthy.txt
Загружен 01_healthy/2807_healthy.txt
470_healthy.txt
Загружен 01_healthy/470_healthy.txt
2355_healthy.txt
Загружен 01_healthy/2355_healthy.txt
image_tumor_patient15012020_633nm_obj20_power100_1sec_acc1_сenter2900_place2_dots20x15_10um_20210209_24.txt
Загружен 01_healthy/image_tumor_patient15012020_633nm_obj20_power100_1sec_acc1_сenter2900_place2_dots20x15_10um_20210209_24.txt


(4944, 1015)

In [10]:
directory = '02_sick'
df_class1, _ = get_data(directory, etalon_columns)
df_class1['labels'] = 1
df_class1.shape

Выходные данные были обрезаны до нескольких последних строк (5000).
2433_sick.txt
Загружен 02_sick/2433_sick.txt
2170_sick.txt
Загружен 02_sick/2170_sick.txt
1914_sick.txt
Загружен 02_sick/1914_sick.txt
170_sick.txt
Загружен 02_sick/170_sick.txt
1779_sick.txt
Загружен 02_sick/1779_sick.txt
2444_sick.txt
Загружен 02_sick/2444_sick.txt
2178_sick.txt
Загружен 02_sick/2178_sick.txt
2150_sick.txt
Загружен 02_sick/2150_sick.txt
1745_sick.txt
Загружен 02_sick/1745_sick.txt
3149_sick.txt
Загружен 02_sick/3149_sick.txt
300_sick.txt
Загружен 02_sick/300_sick.txt
254_sick.txt
Загружен 02_sick/254_sick.txt
1640_sick.txt
Загружен 02_sick/1640_sick.txt
1256_sick.txt
Загружен 02_sick/1256_sick.txt
image_tumor_patient25022020_633nm_obj20_power100_1sec_сenter2900_step4um_place1_20200925_1_13.txt
Загружен 02_sick/image_tumor_patient25022020_633nm_obj20_power100_1sec_сenter2900_step4um_place1_20200925_1_13.txt
1397_sick.txt
Загружен 02_sick/1397_sick.txt
2636_sick.txt
Загружен 02_sick/2636_sick.txt
803_s

(3276, 1015)

In [11]:
df = pd.concat([df_class0, df_class1], axis = 0)

del df_class0, df_class1

In [12]:
y = df['labels']
X = df.drop(['labels'], axis=1)

In [14]:
# Получение ТОП 100 столбцов с максимальными интенсивностями для кажой строки
cols = df.columns
N = 100

# Самое большое значение на последнем месте
top_cols = np.argsort(df.iloc[:, :-1].values, axis=1)[:, -N:]   

# Получение уникальных номеров ТОП столбцов
unique_top_cols =  np.arange(df.shape[1] - 1)

# Создание пустого датафрейма для разреженной матрицы
new_df = pd.DataFrame(index=df.index, columns=unique_top_cols)

# Заполнение датафрейма значениями 
rows = np.arange(len(top_cols)).reshape(-1, 1)

# new_df.values[rows, top_cols] = 1 # Отметим такие частоты 1
new_df.values[rows, top_cols] = np.arange(top_cols.shape[1]) # Нумерованным списком от 0 - N-1, рейтинг частот
new_df = new_df.fillna(0).astype(int)
drop_cols =  new_df.columns[new_df.sum(axis =0) == 0]
new_df = new_df.drop(drop_cols, axis = 1)
new_df.columns = [f'TOP{N} {x}' for x in cols[new_df.columns]]

In [15]:
# Получение 30 столбцов c минимальными интенсивностми для кажой строки
M = 30
down_cols = np.argsort(df.iloc[:, :-1].values, axis=1)[:, :M]   
unique_down_cols = unique_top_cols
new_df2 = pd.DataFrame(index=df.index, columns=unique_down_cols)
rows = np.arange(len(top_cols)).reshape(-1, 1)
new_df2.values[rows, down_cols] = np.arange(down_cols.shape[1])[::-1] # Нумерованным списком от 0 - N-1, рейтинг частот
new_df2 = new_df2.fillna(0).astype(int)
drop_cols =  new_df2.columns[new_df2.sum(axis =0) == 0]
new_df2 = new_df2.drop(drop_cols, axis = 1)
new_df2.columns = [f'D{M} {x}' for x in cols[new_df2.columns]]

In [58]:
a = new_df2.columns.to_list()
b = []
for i in a:
  b.append(i.split()[1])
print(b)
print(len(b))

['2142', '2143', '2145', '2146', '2148', '2150', '2151', '2153', '2154', '2156', '2157', '2159', '2161', '2162', '2164', '2165', '2167', '2168', '2170', '2171', '2173', '2175', '2176', '2178', '2179', '2181', '2182', '2184', '2186', '2187', '2189', '2190', '2192', '2193', '2195', '2196', '2198', '2200', '2201', '2203', '2204', '2206', '2207', '2209', '2210', '2212', '2214', '2215', '2217', '2218', '2220', '2221', '2223', '2224', '2226', '2228', '2229', '2231', '2232', '2234', '2235', '2237', '2238', '2240', '2241', '2243', '2245', '2246', '2248', '2249', '2251', '2252', '2254', '2255', '2257', '2258', '2260', '2262', '2263', '2265', '2266', '2268', '2269', '2271', '2272', '2274', '2275', '2277', '2278', '2280', '2282', '2283', '2285', '2286', '2288', '2289', '2291', '2292', '2294', '2295', '2297', '2298', '2300', '2302', '2303', '2305', '2306', '2308', '2309', '2311', '2312', '2314', '2315', '2317', '2318', '2320', '2321', '2323', '2324', '2326', '2327', '2329', '2331', '2332', '2334',

In [16]:
df = pd.concat([df, new_df, new_df2], axis = 1)
df.shape

(8220, 2648)

In [17]:
y = df['labels']
X = df.drop(['labels'], axis=1)

# Разделение данных на тренировочную, валидационную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, shuffle=True, random_state=STATE)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, shuffle=True, random_state=STATE)

In [18]:
result = {}

In [19]:
# Создание объектов Pool для обучения, валидации и тестирования
train_pool = Pool(data=X_train, label=y_train)
val_pool = Pool(data=X_val, label=y_val)
test_pool = Pool(data=X_test, label=y_test)

In [20]:
# Создание и обучение модели CatBoost с подсчетом метрик

params = {
    'loss_function': 'Logloss',
    'custom_metric': ['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'],
    'eval_metric': 'AUC', #'F1',      #'Accuracy',
    'class_weights': [1, len(y_train) / sum(y_train)],
    'early_stopping_rounds': 200,
    'iterations': 2500,     # Можно еще увеличить
    'learning_rate': 0.03,  # Можно еще уменьшить
    'subsample': 0.8,      # Доля строк для очередного дерева
    'colsample_bylevel': 0.8 # Доля столбцов для очередного дерева
}

catboost_model = CatBoostClassifier(**params)

catboost_model.fit(train_pool, eval_set=val_pool,                    
                   verbose=100, plot = True);


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.8151743	best: 0.8151743 (0)	total: 686ms	remaining: 28m 34s
100:	test: 0.9282042	best: 0.9282042 (100)	total: 49s	remaining: 19m 24s
200:	test: 0.9474589	best: 0.9474589 (200)	total: 1m 33s	remaining: 17m 43s
300:	test: 0.9582494	best: 0.9582494 (300)	total: 2m 14s	remaining: 16m 26s
400:	test: 0.9671398	best: 0.9671398 (400)	total: 3m	remaining: 15m 45s
500:	test: 0.9731672	best: 0.9731672 (500)	total: 3m 43s	remaining: 14m 51s
600:	test: 0.9766010	best: 0.9766010 (600)	total: 4m 26s	remaining: 14m 3s
700:	test: 0.9791777	best: 0.9792282 (699)	total: 5m 12s	remaining: 13m 20s
800:	test: 0.9814701	best: 0.9814701 (800)	total: 5m 56s	remaining: 12m 36s
900:	test: 0.9828499	best: 0.9828692 (897)	total: 6m 48s	remaining: 12m 5s
1000:	test: 0.9838493	best: 0.9838565 (998)	total: 7m 39s	remaining: 11m 28s
1100:	test: 0.9846174	best: 0.9846800 (1087)	total: 8m 26s	remaining: 10m 43s
1200:	test: 0.9853061	best: 0.9853495 (1184)	total: 9m 14s	remaining: 9m 59s
1300:	test: 0.9857444	

In [21]:
# Предсказание на тестовом наборе
y_pred = catboost_model.predict(test_pool)
y_pred_prob = catboost_model.predict_proba(test_pool)[: ,1]

# Отобразим результаты модели
res = model_metric(y_test, y_pred, y_pred_prob)
result['catboost'] = res

Accuracy: 0.937
Precision: 0.890
Recall: 0.962
F1-score: 0.924


In [22]:
import lightgbm as lgb

# Создание объектов Dataset для обучения, валидации и тестирования
train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val)
test_data = lgb.Dataset(X_test, label=y_test)

# Параметры модели LightGBM
params = {
    'objective': 'binary',
    #'metric': ['binary_logloss', 'accuracy', 'precision', 'recall', 'f1', 'auc'],
    'is_unbalance': True,
    'learning_rate': 0.02,
    'feature_fraction': 0.75,    
    'bagging_fraction': 0.75, 
    'bagging_freq': 5,
    # 'lambda_l1': 10,
    # 'lambda_l2': 10
}

evals_result ={}
# Обучение модели LightGBM
model_lgb = lgb.train(params, train_data,
                  num_boost_round=1500,
                  valid_sets=[train_data, val_data],
                  early_stopping_rounds=200,
                  evals_result=evals_result,
                  verbose_eval=100)

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:260: UserWarning: 'evals_result' argument is deprecated and will be removed in a future release of LightGBM. Pass 'record_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'evals_result' argument is deprecated an

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2096, number of negative: 3164
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.366016 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 310344
[LightGBM] [Info] Number of data points in the train set: 5260, number of used features: 1984
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.398479 -> initscore=-0.411806
[LightGBM] [Info] Start training from score -0.411806
Training until validation scores don't improve for 200 rounds
[100]	training's binary_logloss: 0.263619	valid_1's binary_logloss: 0.313341
[200]	training's binary_logloss: 0.163814	valid_1's binary_logloss: 0.242885
[300]	training's binary_logloss: 0.108159	valid_1's binary_logloss: 0.200872
[400]	training's binary_logloss: 0.0730839	valid_

In [41]:
!pip show lightgbm

Name: lightgbm
Version: 3.3.5
Summary: LightGBM Python Package
Home-page: https://github.com/microsoft/LightGBM
Author: 
Author-email: 
License: The MIT License (Microsoft)
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, scikit-learn, scipy, wheel
Required-by: 


In [23]:
# Предсказание на тестовом наборе
y_pred_prob = model_lgb.predict(X_test)
y_pred =  1*(y_pred_prob>.5)

# Отобразим результаты модели
res = model_metric(y_test, y_pred, y_pred_prob)
result['lightgbm'] = res

Accuracy: 0.948
Precision: 0.924
Recall: 0.947
F1-score: 0.935


In [24]:
torch.manual_seed(STATE)
torch.cuda.manual_seed(STATE)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [34]:
class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        
        self.model = nn.Sequential(
            nn.Linear(input_size, 1536),
            nn.Dropout(0.5),
            nn.ReLU(),
            
            nn.Linear(1536, 1024),
            nn.Dropout(0.5),
            nn.ReLU(),

            nn.Linear(1024, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            
            nn.Linear(256, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),

            nn.Linear(32, output_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [35]:
# Создание тензоров
X_train_torch = torch.Tensor(X_train.values).to(device)
y_train_torch = torch.Tensor(y_train.values).to(device).view(-1, 1)

X_val_torch = torch.Tensor(X_val.values).to(device)
y_val_torch = torch.Tensor(y_val.values).to(device).view(-1, 1)

X_test_torch = torch.Tensor(X_test.values).to(device)
y_test_torch = torch.Tensor(y_test.values).to(device).view(-1, 1)

In [36]:
# Инициализация модели, функции потерь и оптимизатора
input_size=X_train_torch.shape[1]
model_net = Net(input_size=input_size, output_size=1).to(device)

In [46]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model_net.parameters(), lr=0.005)

# Списки для сохранения и отображения лосса
train_losses = []
val_losses = []
best_val_loss = float('inf')  
best_state_dict ={}

# Обучение модели
num_epochs = 5000
batch_size = 2048
num_batches = int(np.ceil(X_train.shape[0] / batch_size))

for epoch in tqdm(range(num_epochs)):
    # Перемешивание данных в каждую эпоху
    indices = torch.randperm(X_train.shape[0])
    X_train_epoch = X_train_torch[indices]
    y_train_epoch = y_train_torch[indices]

    train_batch_losses = []
    val_batch_losses = []
    
    # Обучение на батчах
    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = (batch_idx + 1) * batch_size
        X_batch = X_train_torch[start_idx:end_idx]
        y_batch = y_train_torch[start_idx:end_idx]

        # Прогоняем данные через модель
        outputs = model_net(X_batch)

        # Вычисляем функцию потерь
        loss = criterion(outputs, y_batch)

        # Обновляем веса
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Сохраняем лосс на трейне
        #train_losses.append(loss.item())
        train_batch_losses.append(loss.item())

        # Вычисление лосса на валидации
        with torch.no_grad():
            val_outputs = model_net(X_val_torch)
            val_loss = criterion(val_outputs, y_val_torch)
            #val_losses.append(val_loss.item())
            val_batch_losses.append(val_loss.item())
            
            if val_loss < best_val_loss:
                best_val_loss = val_loss            
                best_state_dict = model_net.state_dict()
    train_losses.append(np.median(train_batch_losses))    
    val_losses.append(np.median(val_batch_losses))
    if epoch % 100 == 0:
        print(f"Epoch {epoch + 1}, Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}")

# Визуализация изменения лосса на трейне и валидации
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Val Loss")
plt.legend()
plt.show()

  0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 1, Train Loss: 0.5075, Val Loss: 0.5085
Epoch 101, Train Loss: 0.4795, Val Loss: 0.5057
Epoch 201, Train Loss: 0.4987, Val Loss: 0.4935
Epoch 301, Train Loss: 0.4801, Val Loss: 0.4940
Epoch 401, Train Loss: 0.4740, Val Loss: 0.4720
Epoch 501, Train Loss: 0.4337, Val Loss: 0.4562


KeyboardInterrupt: ignored

In [ ]:
# Предскажем на тестовом наборе лучшим экземпляром сети
best_model = Net(input_size=input_size, output_size=1).to(device)
best_model.load_state_dict(best_state_dict)

y_pred_prob = best_model(X_test_torch).detach().cpu()
y_pred = (y_pred_prob > 0.5).int()

# Отобразим результаты модели
res = model_metric(y_test, y_pred, y_pred_prob)
result['perceptron3'] = res

In [ ]:
!mkdir data
!cp 01_healthy data/
!cp 02_sick data/

In [ ]:
root_folder = "data"
all_files = {}
for fld in next(os.walk(root_folder))[1]:
  path = os.path.join(root_folder, fld)
  all_files[fld] = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

In [ ]:
measurements = []
classifications = []
count = 0
for sample in list(all_files):
  for txt in all_files[sample]:
    df = pd.read_csv(
        os.path.join(root_folder,sample,txt),
        sep='\t',skiprows=[0], header=None, 
        names=['Wave', 'Intensity']
    )
    if len(df) == 1014:
      df = pd.read_csv(
        os.path.join(root_folder,sample,txt),
        sep='\t', header=None, 
        names=['Wave', 'Intensity']
      )
    # print(str(txt), df['Intensity'].to_numpy().shape[0])
    if df['Intensity'].to_numpy().shape[0] == 1015:
      measurements.append(df['Intensity'].to_numpy())
      # print(True, end='')
    # print(df['Intensity'].to_numpy().shape, str(txt))
      if str(sample) == '01_healthy':
        classifications.append([0])
      if str(sample) == '02_sick':
        classifications.append([1])
      count+=1
# print(measurements)
X_train = np.asarray(measurements)
Y_train = np.asarray(classifications)
print(count)

In [ ]:
measurements = []
classifications = []
count = 0
for sample in list(all_files):
  for txt in all_files[sample]:
    df = pd.read_csv(
        os.path.join(root_folder,sample,txt),
        sep='\t',skiprows=[0], header=None, 
        names=['Wave', 'Intensity']
    )
    if len(df) == 1014:
      df = pd.read_csv(
        os.path.join(root_folder,sample,txt),
        sep='\t', header=None, 
        names=['Wave', 'Intensity']
      )
    # print(str(txt), df['Intensity'].to_numpy().shape[0])
    if df['Intensity'].to_numpy().shape[0] == 1015:
      measurements.append(df['Intensity'].to_numpy())
      # print(True, end='')
    # print(df['Intensity'].to_numpy().shape, str(txt))
      if str(sample) == '01_healthy':
        classifications.append([0])
      if str(sample) == '02_sick':
        classifications.append([1])
      count+=1
# print(measurements)
X_train = np.asarray(measurements)
Y_train = np.asarray(classifications)
print(count)

In [ ]:
X_train = X_train.reshape(count, 1015)
Y_train[Y_train == 0].shape

In [ ]:
tmp = baseline_als(X_train[0], 10000, 0.009)
X_train_baselineless = np.array([baseline_als(i, 1000, 0.007) for i in X_train])
X_train_baselineless = X_train_baselineless - X_train_baselineless.min(axis=1).reshape((-1, 1))
X_train = (X_train - X_train.mean(axis=1)[:, None]) / X_train.std(axis=1)[:, None]

X_train_baselineless = (X_train_baselineless - X_train_baselineless.mean(axis=1)[:, None]) / X_train_baselineless.std(axis=1)[:, None]

xtrain, xtest, ytrain, ytest = train_test_split(X_train, Y_train, test_size=0.2, shuffle=True)


In [ ]:
model1 = Sequential()
model1.add(Conv1D(512, 2, activation='relu', input_shape=(1015,1)))
model1.add(MaxPooling1D())
model1.add(Flatten())
model1.add(Dense(512, activation = 'relu'))
model1.add(Dense(1, activation = 'sigmoid'))
model1.summary()
model1.compile(
    loss=BinaryCrossentropy(), 
    optimizer='adam', 
    metrics=[Precision(thresholds=0.5), Recall(thresholds=0.5), F1Score(num_classes=1), AUC(), 'accuracy']
)
model1_hist = model1.fit(xtrain, ytrain, batch_size=64, epochs=20, verbose=1)

pred = model1.predict(xtest)

In [ ]:
df_result = pd.DataFrame.from_dict(result, orient='index')
df_result = df_result.sort_values(by='roc_auc', ascending=False)
df_result